# ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

<img src="https://github.com/GoogleCloudPlatform/agent-starter-pack/blob/main/docs/images/adk_logo.png?raw=true" width="400">



## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform --upgrade

### Import libraries

In [1]:
import json

import requests
import vertexai.agent_engines

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [2]:
# Replace with your Agent Engine ID
AGENT_ENGINE_ID = "projects/168027251845/locations/us-central1/reasoningEngines/968489424163700736"

In [3]:
remote_agent_engine = vertexai.agent_engines.get(AGENT_ENGINE_ID)

In [4]:
for event in remote_agent_engine.stream_query(message="hi!", user_id="test"):
    print(event)

{'content': {'parts': [{'text': 'Hello! How can I help you today?\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 10, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 10}], 'prompt_token_count': 147, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 147}], 'total_token_count': 157, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-61e7a278-dfbd-43cd-a8c4-4d3cff21e177', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '79bba3c3-1917-425f-923d-6c6bb9f6aa40', 'timestamp': 1757215325.879098}


In [5]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [1]:
# Uncomment the following lines if you're not using the virtual environment created by uv
import sys
import json
sys.path.append("../")

In [2]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [3]:
for event in agent_engine.stream_query(message="what are the nearby starbucks stores to my toronto stores?", user_id="test"):
    print(json.dumps(event, indent=2))

{
  "content": {
    "parts": [
      {
        "thought_signature": "CqcDAcu98PB9pDYUtw-RV_y2DzURqdDxt95LbDHH9AB5Y5RN6Ii8m12at0-wylsdc-HkrYUx5QS98s5tYmm-46FYkbW-1n_s23wyFc6r0WWSx4ul9pp75lKskh5pEiTrSGcb2eLkYeuojfSj0fzI9lrNgFz1UL7nQQXcS1KiwBIV9ydTf-TB7luxrldGQM_Wt3EFUOgMB3H7QNNAiPbnp-1aQzyrU3j85YVb1HTBFjBOyk2qV0698F8f7jqOD265pbqyVX1zEqdag6pNPePBHDj1Eu8fN8C3HPDnp6cBw-E4li4kD8Wj-3Uc7BLAubn2NCSfE4Dx-iuJKzc02HuKZz5TRhPo3rmoTwInEJFO_MTI08WHdYVm7fQa5nQjWzqUfwkq6LEN-bFmG7TttSEAcCYEAIPNhUkwLRumP5I9NWUJ9ERdOjn4ul7LtDqw3cPFJY4MgkeWAhXXEZQ2VHKCpG2qTvKtTRuk9vm1w0cstzA-CxZVjFk7pQqKwo1cblNok5bE_mcOa2zUjEgYfmqFoyNY9OVtX-1q2H0-pI2JMv__UxwqqoSlUDVP",
        "function_call": {
          "id": "adk-10be61c4-ffe8-48db-901c-97da5bb6b988",
          "args": {
            "user_query": "what are the nearby starbucks stores to my toronto stores?"
          },
          "name": "route_intent"
        }
      }
    ],
    "role": "model"
  },
  "usage_metadata": {
    "candidates_token_count": 17,
    "candidat

## If you are using Cloud Run

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [20]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL

You'll need to first create a Session

In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

Then you will be able to send a message

In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

### Local Testing

> You can run the application locally via the `make local-backend` command.

#### Create a session
 Create a new session with user preferences and state information


In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"http://127.0.0.1:8000/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

#### Send a message
Send a message to the backend service and receive a streaming response


In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = "http://127.0.0.1:8000/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")